# Inference

In [1]:
import numpy as np
import random
import os
import math
from itertools import product

from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler


from glob import glob
import pandas as pd
import cv2
from tqdm.auto import tqdm
from PIL import Image
from pathlib import Path
import matplotlib.pyplot as plt


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
# from torch.utils.tensorboard import SummaryWriter

from torch.autograd import Variable

import torchvision.models as models
from torchvision import transforms

import albumentations as A
import albumentations.pytorch

from sklearn.metrics import accuracy_score

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using {device} device")

Using cpu device


In [3]:
import tensorflow as tf
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

In [4]:
CFG = {
    'IMG_SIZE':256,
    'EPOCHS':50,
    'PATIENCE':10,
#     'class':41 # quasi_drug
    'class':126 # home_clean
    # 'class':41 # HMR 
}

In [5]:
os.listdir('./data/home_clean')

['RegNet_0.0001_rmsprop_CosineAnnealing_example.pth',
 'RegNet_0.001_rmsprop_CosineAnnealing_example (2).pth',
 'RegNet_1e-05_rmsprop_CosineAnnealing_example.pth']

In [6]:
# Train_path = '/home/lab17/jupyter_home/Data/product_image/Training/'
# Valid_path = '/home/lab17/jupyter_home/Data/product_image/Validation/'
# Test_path = '/home/lab17/jupyter_home/git/test_img/'
# save_graph_path = './result/'
# save_model_path = '/home/lab17/jupyter_home/saved_models/'

In [7]:
Train_path = './data/home_clean_dataset'
Test_path = 'C:/jupyter_home/GIT/mulcam3projcet_ai/test_img/home-clean'

# Load Data

### train

In [20]:
def get_train_data(data_dir):
#     img_path_list = []
    label_list = []
    label_name_list = []
    
#     image_path = os.path.join(data_dir, 'dessert')
    image_path = data_dir
    
    for product_name in os.listdir(image_path):
        product_path = os.path.join(image_path, product_name)
        if os.path.isdir(product_path):
#             get image path
#             img_path_list.extend(glob(os.path.join(product_path, '*.jpg')))
#             img_path_list.extend(glob(os.path.join(product_path, '*.png')))
            label = list(product_name.split('_')[0])
            name = product_name[6:]
            
            # get label
            label_list.append(''.join(label))
            label_name_list.append(name)
                
#     return img_path_list, label_list
    return label_list, label_name_list

In [21]:
# def get_train_data(data_dir):
#     img_path_list = []
#     label_list = []
    
# #     image_path = os.path.join(data_dir, dir_name)
#     image_path = data_dir
    
#     for product_name in os.listdir(image_path):
#         product_path = os.path.join(image_path, product_name)
#         if os.path.isdir(product_path):
#             # get image path
#             img_path_list.extend(glob(os.path.join(product_path, '*.jpg')))
#             img_path_list.extend(glob(os.path.join(product_path, '*.png')))
# #             label = list(product_name[:5])
            
#             label = list(product_name.split('_')[0])
            
#             # get label
#             label_list.append(''.join(label))
                
#     return img_path_list, label_list

In [22]:
label_list, label_name_list = get_train_data(Train_path)

In [23]:
# encoder
le = preprocessing.LabelEncoder()
targets = le.fit_transform(label_list)
print('--targets\n' , targets)

label_encoder = {key:val for key, val in zip(label_list, targets)}
print(label_encoder)

--targets
 [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125]
{'10182': 0, '10221': 1, '10222': 2, '15285': 3, '15344': 4, '15347': 5, '15351': 6, '15352': 7, '15353': 8, '15355': 9, '15358': 10, '15359': 11, '15361': 12, '15362': 13, '15363': 14, '15366': 15, '15369': 16, '15399': 17, '15854': 18, '15857': 19, '15860': 20, '15871': 21, '20192': 22, '20209': 23, '20215': 24, '20273': 25, '25016': 26, '25297': 27, '25351': 28, '25369': 29, '25371': 30, '25783': 31, '30146': 32, '30168': 33, '30182': 34, '30193': 35, '30225': 36, '3024

In [24]:
# decoder
label_decoder = {v: k for k, v in label_encoder.items()}
label_name_decoder = {key : value for key, value in zip(label_list, label_name_list)}

### validation

In [25]:
# def get_valid_data(data_dir):
#     img_valid_list = []
#     label_valid_list = []
    
#     image_path = os.path.join(data_dir, 'dessert')
    
#     for product_name in os.listdir(image_path):
#         product_path = os.path.join(image_path, product_name)
#         if os.path.isdir(product_path):
#             # get image path
#             img_valid_list.extend(glob(os.path.join(product_path, '*.jpg')))
#             img_valid_list.extend(glob(os.path.join(product_path, '*.png')))
#             label = list(product_name[:5])
            
#             # get label
#             label_valid_list.append(''.join(label))
                
#     return img_valid_list, label_valid_list

In [26]:
# def valid_data_blanced(img, label):
#     x = []
#     y = []
    
#     for i in range(CFG['class']):
#         _img = img[(i * 15): ((i + 1) * 15)]
#         _label = label[i]
        
#         for img_product in _img:
#             x.append(img_product)
#             y.append(_label)
            
#     return x, y

In [27]:
# img_valid_list, label_valid_list = get_valid_data(Valid_path)
# x_valid, y_valid = valid_data_blanced(img_valid_list, label_valid_list)
# len(label_valid_list)

In [28]:
# le2 = preprocessing.LabelEncoder()
# targets_y = le2.fit_transform(y_valid)
# targets_y_t = torch.as_tensor(targets_y)
# one_hot_valid_y = F.one_hot(targets_y_t)
# one_hot_valid_y.shape

### test

In [29]:
def get_test_data(data_dir):
    img_path_list = []
    label_list = []
    
    image_path = data_dir
    
#     for product in os.listdir(image_path):

    # get image path
    img_path_list.extend(glob(os.path.join(image_path, '*.jpg')))
    img_path_list.extend(glob(os.path.join(image_path, '*.png')))
    label_list = [ip[len(Test_path):].split('_')[0][1:] for ip in img_path_list]
    # get label
#     label_list.append(''.join(label))
                
    return img_path_list, label_list

In [30]:
test_img_path, test_label_list = get_test_data(Test_path)

In [31]:
print(len(test_label_list))

150


In [32]:
# test data
test_x = test_img_path
# 레이블을 one-hot-vector로 변환
test_y = [label_encoder[key] for key in test_label_list]
test_targets = torch.as_tensor(test_y)
one_hot_test_y = F.one_hot(test_targets)

In [33]:
print([f'{i}, {y}' for i, y in zip(test_label_list, test_y)])

['10182, 0', '10182, 0', '10221, 1', '10222, 2', '10222, 2', '15285, 3', '15347, 5', '15347, 5', '15351, 6', '15351, 6', '15352, 7', '15353, 8', '15355, 9', '15355, 9', '15359, 11', '15359, 11', '15361, 12', '15361, 12', '15362, 13', '15363, 14', '15366, 15', '15369, 16', '15369, 16', '15399, 17', '15399, 17', '15854, 18', '15854, 18', '15860, 20', '15871, 21', '20192, 22', '20215, 24', '20273, 25', '25016, 26', '25297, 27', '25351, 28', '25371, 30', '30168, 33', '30182, 34', '30193, 35', '30225, 36', '30250, 38', '35295, 39', '35308, 41', '35309, 42', '35309, 42', '35310, 43', '35315, 44', '35316, 45', '35316, 45', '35318, 46', '35323, 47', '35323, 47', '35327, 48', '35329, 49', '35329, 49', '35331, 51', '35331, 51', '35332, 52', '35332, 52', '35336, 54', '35336, 54', '35343, 55', '35345, 56', '35347, 57', '35545, 58', '35545, 58', '35546, 59', '35546, 59', '35757, 60', '40168, 62', '45023, 64', '45307, 65', '45309, 66', '45313, 67', '45314, 68', '45318, 70', '45318, 70', '45318, 70',

# Model

In [34]:
class AlbumentationsCustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, train_mode=True, transforms=None):
        self.transforms = transforms
        self.train_mode = train_mode
        self.img_path_list = img_path_list
        self.label_list = label_list

    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        # Get image data
        image = cv2.imread(img_path)
        
        # By default OpenCV uses BGR color space for color images,
        # so we need to convert the image to RGB color space.
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.train_mode:
#             image = image.astype(np.int16)
            augmented = self.transforms(image=image)
            image = augmented['image']
            label = self.label_list[index]
            return image, label
        else:
            augmented = self.transforms(image=image)
            image = augmented['image']
            label = self.label_list[index]
            return image, label
    
    def __len__(self):
        return len(self.img_path_list)
    
A_test_transform = albumentations.Compose([
                                    A.Resize(256, 256),
#                                     A.Normalize(mean=(0.744859, 0.735139, 0.711357), std=(0.100712, 0.120692, 0.167998)),  # dessert
                                    A.Normalize(mean=(0.719834, 0.727319, 0.72394), std=(0.135879, 0.130642, 0.137573)),   # quasi_drug
#                                     A.pytorch.transforms.ToTensor(),
                                    A.pytorch.transforms.ToTensorV2(transpose_mask=True),
                                ])

# A_vali_dataset = AlbumentationsCustomDataset(x_valid, one_hot_valid_y, train_mode=True, transforms=A_test_transform)
# A_vali_loader = DataLoader(A_vali_dataset, batch_size = 5, shuffle=False, num_workers=0, collate_fn=None)

A_test_dataset = AlbumentationsCustomDataset(test_x, one_hot_test_y, train_mode=False, transforms=A_test_transform)
A_test_loader = DataLoader(A_test_dataset, batch_size = 4, shuffle=False, num_workers=0, collate_fn=None)

In [35]:
class ResNet50(torch.nn.Module):
    def __init__(self):
        super(ResNet50, self).__init__()
        model = models.resnet50(pretrained=True)
        modules = list(model.children())[:-1]
        self.feature_extract = nn.Sequential(*modules)
        self.fc1 = nn.Linear(2048, 1000)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(1000,CFG['class'])
#         self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.feature_extract(x)
        # x = x.mean(dim=(-2, -1))
        # (batch, 2048, 4, 4)
        x = torch.squeeze(x)
        x = self.relu(self.fc1(x))
        out = self.fc2(x)
#         out = self.softmax(x)
        
        return out

In [36]:
class EfficientNetb4(torch.nn.Module):
    def __init__(self):
        super(EfficientNetb4, self).__init__()
        model = models.efficientnet_b4(pretrained=True)
        modules = list(model.children())[:-1]
        self.feature_extract = nn.Sequential(*modules)
        self.fc1 = nn.Linear(1792, 1000)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(1000, CFG['class'])
#         self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.feature_extract(x)
        # (batch, 1792, 1, 1)
        x = torch.squeeze(x)
        x = self.relu(self.fc1(x))
        out = self.fc2(x)
#         out = self.softmax(x)
        
        return out

In [37]:
class RegNet(torch.nn.Module):
    def __init__(self):
        super(RegNet, self).__init__()
        model = models.regnet_y_16gf(pretrained=True)
        modules = list(model.children())[:-1]
        self.feature_extract = nn.Sequential(*modules)
        self.fc1 = nn.Linear(3024, 1000)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(1000, CFG['class'])
#         self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.feature_extract(x)
        # (batch, 3024, 1, 1)
        x = torch.squeeze(x)
        x = self.relu(self.fc1(x))
        out = self.fc2(x)
#         out = self.softmax(x)
        
        return out

In [38]:
def score_function(real, pred):
    score = accuracy_score(real, pred)
    return score

# Inference2

In [39]:
# # sorted(os.listdir('/home/lab17/jupyter_home/saved_models/'))
# file_list = sorted(glob('/home/lab17/jupyter_home/saved_models/*.pth'))
# print(len(file_list))
# sorted(glob('/home/lab17/jupyter_home/saved_models/*.pth'))

In [40]:
# file_list=['/home/lab17/jupyter_home/saved_models/RegNet_0.001_rmsprop_CosineAnnealing_example.pth']

In [39]:
# model_dir = './data/quasi_drug/'

In [43]:
model_dir = './data/home_clean/'

In [44]:
os.listdir(model_dir)

['RegNet_0.0001_rmsprop_CosineAnnealing_example.pth',
 'RegNet_0.001_rmsprop_CosineAnnealing_example (2).pth',
 'RegNet_1e-05_rmsprop_CosineAnnealing_example.pth']

In [45]:
file_list = sorted(glob(os.path.join(model_dir,'*.pth') ))
file_list

['./data/home_clean\\RegNet_0.0001_rmsprop_CosineAnnealing_example.pth',
 './data/home_clean\\RegNet_0.001_rmsprop_CosineAnnealing_example (2).pth',
 './data/home_clean\\RegNet_1e-05_rmsprop_CosineAnnealing_example.pth']

In [46]:
pred_ensemble = []
batch_size = 34

device = torch.device('cpu')

for m_path in file_list:
    if 'ResNet50' in m_path:
        model_test = ResNet50().to(device)
    elif 'EfficientNetb4' in m_path:
        model_test = EfficientNetb4().to(device)
    elif 'RegNet' in m_path:
        model_test = RegNet().to(device)    

    model_test.load_state_dict(torch.load(m_path, map_location=device))
    model_test.eval()

    criterion = torch.nn.CrossEntropyLoss()

    test_loss = []
    f_pred = []

#     for img, label in tqdm(iter(A_vali_loader)):
#         img, label = img.float().to(device), label.float().to(device)

#         # Data -> Model -> Output
#         logit = model_test(img)
#         logit = torch.squeeze(logit)

#         # Calc loss
#         loss = criterion(logit, label)

#         test_loss.append(loss.item())
#         f_pred.extend(logit.argmax(1).detach().cpu().numpy().tolist())

#     print('val loss :' ,np.mean(test_loss))
#     print('val acc :', score_function(targets_y, f_pred))

#     test_loss = []
#     f_pred = []

    for img, label in tqdm(iter(A_test_loader)):
        img, label = img.float().to(device), label.float().to(device)

        # Data -> Model -> Output
        logit = model_test(img)
        logit = torch.squeeze(logit)

        # Calc loss
        loss = criterion(logit, label)

        test_loss.append(loss.item())
        f_pred.extend(logit.argmax(1).detach().cpu().numpy().tolist())
        
        
    print(m_path[len(model_dir)+2:])

    print('test loss :' ,np.mean(test_loss))
    print('test acc :', score_function(test_y, f_pred))

  0%|          | 0/38 [00:00<?, ?it/s]

gNet_0.0001_rmsprop_CosineAnnealing_example.pth
test loss : 2.2906102000882753
test acc : 0.5266666666666666


  0%|          | 0/38 [00:00<?, ?it/s]

gNet_0.001_rmsprop_CosineAnnealing_example (2).pth
test loss : 10.496223060708298
test acc : 0.26


  0%|          | 0/38 [00:00<?, ?it/s]

gNet_1e-05_rmsprop_CosineAnnealing_example.pth
test loss : 1.5618299064471532
test acc : 0.6533333333333333


In [47]:
def print_ox(test_y,f_pred):
    return 'o' if test_y==f_pred else 'x'
def print_name(test_y, f_pred):
    return f'{label_decoder[y]}, {label_name_decoder[label_decoder[y]]} : {label_name_decoder[label_decoder[p]]}' if test_y!=f_pred else ''
print('o/x | onehot test, pred num | img_num : pred_img_name\n')
for y, p in zip(test_y, f_pred):
    print(f'{print_ox(y,p)} | {y}, {p} | {print_name(y,p)}')
    
    
# print(f'{print_ox(y,p)} | {y}, {p} | {label_decoder[y]} : {label_name_decoder[label_decoder[p]]}')    
#     print(f'{print_ox(y,p)} {y} : {p} {label_name_decoder[y]}, {label_name_decoder[p]}')
# print(f'{test_y}, {f_pred}')
# print(f'{print_ox(test_y,f_pred)} {test_y} : {f_pred} {label_name_decoder[test_y]}, {label_name_decoder[f_pred]}')

o/x | onehot test, pred num | img_num : pred_img_name

o | 0, 0 | 
o | 0, 0 | 
o | 1, 1 | 
x | 2, 50 | 10222, 슈가버블친환경주방세제470ML : 슈가버블)친환경주방세제리치레몬
o | 2, 2 | 
x | 3, 74 | 15285, 엘지베비언스핑크퐁세탁세제2.2L : 유한양행)유한락스도마행주용
o | 5, 5 | 
o | 5, 5 | 
o | 6, 6 | 
o | 6, 6 | 
o | 7, 7 | 
x | 8, 27 | 15353, 유한양행유한젠1.4KG : 무궁화왕표백450G
o | 9, 9 | 
x | 9, 107 | 15355, 유한양행암앤해머베이킹소다2.1KG : 하수구냄새차단트랩
o | 11, 11 | 
x | 11, 56 | 15359, 애경트리오곡물설거지우리쌀겨1.2L : 피죤)피죤옐로미모사
o | 12, 12 | 
o | 12, 12 | 
o | 13, 13 | 
o | 14, 14 | 
o | 15, 15 | 
x | 16, 71 | 15369, 유한양행유한락스흰옷표백용1L : 헨켈)고농축버넬아몬드블러섬
o | 16, 16 | 
x | 17, 85 | 15399, 피죤무균무때주방용 : LG퐁퐁친환경주방세제오렌지
x | 17, 67 | 15399, 피죤무균무때주방용 : LG생활건강)테크울드라이오리지널
o | 18, 18 | 
o | 18, 18 | 
x | 20, 61 | 15860, 슈가버블식기세척기용헹굼보조제1L : 자연미인로즈비누
o | 21, 21 | 
o | 22, 22 | 
o | 24, 24 | 
x | 25, 67 | 20273, 불스원퍼스트클래스컴파운드 : LG생활건강)테크울드라이오리지널
x | 26, 27 | 25016, 무균무때욕실그린허브500ML : 무궁화왕표백450G
x | 27, 105 | 25297, 무궁화왕표백450G : 애경)스파크1KG
o | 28, 28 | 
o | 30, 30 | 
x | 33, 104 | 30168, 요리지)크